In [5]:
from dash import Dash, html, dcc, Input, Output
import flask
#from sigfig import round
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import plot
import numpy as np
import pandas as pd
import zipfile

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Pegando os dados em zip
dados_prf_2018 = pd.read_csv('https://arquivos.prf.gov.br/arquivos/index.php/s/MaC6cieXSFACNWT/download',
                        encoding = 'latin1', compression = 'zip', sep = ';')
dados_prf_2019 = pd.read_csv('https://arquivos.prf.gov.br/arquivos/index.php/s/kRBUylqz6DyQznN/download',
                        encoding = 'latin1', compression = 'zip', sep = ';')
dados_prf_2020 = pd.read_csv('https://arquivos.prf.gov.br/arquivos/index.php/s/rVfIQjF0wrwHa5P/download',
                        encoding = 'latin1', compression = 'zip', sep = ';')
dados_prf_2021 = pd.read_csv('http://arquivos.prf.gov.br/arquivos/index.php/s/n1T3lymvIdDOzzb/download',
                        encoding = 'latin1', compression = 'zip', sep = ';')

# juntando todo os dataframes com concat
juntando_dados = pd.concat([dados_prf_2018,dados_prf_2019 ,dados_prf_2020,dados_prf_2021],
                          keys = ['2018', '2019', '2020', '2021'])
juntando_dados

id data_inversa     dia_semana   horario  uf     br     km  \
2018 0      100027.0   2018-01-01  segunda-feira  06:00:00  TO  153.0  393,1   
     1      100044.0   2018-01-01  segunda-feira  08:30:00  SE  101.0  123,4   
     2      100046.0   2018-01-01  segunda-feira  08:50:00  TO  153.0    347   
     3      100052.0   2018-01-01  segunda-feira  05:30:00  PA  155.0    221   
     4      100053.0   2018-01-01  segunda-feira  10:25:00  BA  116.0  289,6   
...              ...          ...            ...       ...  ..    ...    ...   
2021 64534  465833.0   2021-11-17   quarta-feira  19:00:00  MG  262.0  451,3   
     64535  465953.0   2021-10-27   quarta-feira  18:20:00  MG  116.0  706,4   
     64536  465955.0   2021-10-28   quinta-feira  08:50:00  RJ  116.0  169,5   
     64537  465965.0   2021-11-07        domingo  20:15:00  SP  116.0    318   
     64538  465993.0   2021-08-25   quarta-feira  15:10:00  RO  364.0    970   

                      municipio  \
2018 0             RIO DOS BOIS   
     1        ITAPORANGA DAJUDA   
     2                   GUARAI   
     3      ELDORADO DO CARAJAS   
     4                   TUCANO   
...                         ...   
2021 64534         NOVA SERRANA   
     64535               MURIAE   
     64536   SAO JOAO DE MERITI   
     64537            JUQUITIBA   
     64538          PORTO VELHO   

                                               causa_acidente  \
2018 0                            Falta de Atenção à Condução   
     1                     Não guardar distância de segurança   
     2                            Falta de Atenção à Condução   
     3                                     Ingestão de Álcool   
     4                                 Ultrapassagem Indevida   
...                                                       ...   
2021 64534                           Transitar no acostamento   
     64535  Acessar a via sem observar a presença dos outr...   
     64536  Condutor deixou de manter distância do veículo...   
     64537                           Transitar no acostamento   
     64538  Acessar a via sem observar a presença dos outr...   

                            tipo_acidente  ... feridos_graves ilesos  \
2018 0                    Colisão frontal  ...              1      1   
     1                   Colisão traseira  ...              0      2   
     2                    Colisão lateral  ...              0      2   
     3                    Colisão lateral  ...              0      1   
     4          Saída de leito carroçável  ...              1      0   
...                                   ...  ...            ...    ...   
2021 64534            Colisão transversal  ...              0      1   
     64535  Colisão lateral mesmo sentido  ...              0      1   
     64536  Colisão lateral mesmo sentido  ...              0      1   
     64537      Atropelamento de Pedestre  ...              2      0   
     64538               Colisão traseira  ...              3      1   

           ignorados feridos veiculos      latitude     longitude  regional  \
2018 0             0       1        2   -9,35641407  -48,54610294   SPRF-TO   
     1             0       0        2  -11,04476007     -37,36498   SPRF-SE   
     2             0       0        2     -8,952295    -48,500107   SPRF-TO   
     3             0       0        2     -6,345512    -49,389626   SPRF-PA   
     4             0       1        1  -11,08661652  -38,83420181   SPRF-BA   
...              ...     ...      ...           ...           ...       ...   
2021 64534         0       1        2  -19,86372828  -44,99796931   SPRF-MG   
     64535         2       2        2    -21,147048    -42,381522   SPRF-MG   
     64536         0       1        2    -22,816206    -43,324463   SPRF-RJ   
     64537         0       2        1  -23,92536222  -46,99376156   SPRF-SP   
     64538         0       3        2   -9,65616271  -65,72389011   SPRF-RO   

            delegacia               uop  

In [4]:
# quantos acidentes ocorreram em cada ano.

acidentes_2018 = len(dados_prf_2018.index)
acidentes_2019 = len(dados_prf_2019.index)
acidentes_2020 = len(dados_prf_2020.index)
acidentes_2021 = len(dados_prf_2021.index)
data = {
    'Anos': ['2018', '2019', '2020', '2021'],
    'Acidentes': [acidentes_2018,acidentes_2019,acidentes_2020,acidentes_2021]}
acidentes_ano = pd.DataFrame(data)
acidentes_ano


,Anos,Acidentes
0,2018,69332
1,2019,67556
2,2020,63576
3,2021,64539


In [5]:
opcoes = juntando_dados.uf.unique()

In [6]:
opcoes = list(dados_prf_2018.uf.unique())


In [7]:
acidentes_pb_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'PB') >= 1 ])
acidentes_pb_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'PB') >= 1 ])
acidentes_pb_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'PB') >= 1 ])
acidentes_pb_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'PB') >= 1 ])
acidentes_pb_todos = len(juntando_dados[(juntando_dados.uf == 'PB') >= 1 ])


acidentes_rj_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'Rj') >= 1 ])
acidentes_rj_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'RJ') >= 1 ])
acidentes_rj_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'RJ') >= 1 ])
acidentes_rj_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'RJ') >= 1 ])
acidentes_rj_todos = len(juntando_dados[(juntando_dados.uf == 'RJ') >= 1 ])


acidentes_sc_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'SC') >= 1 ])
acidentes_sc_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'SC') >= 1 ])
acidentes_sc_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'SC') >= 1 ])
acidentes_sc_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'SC') >= 1 ])
acidentes_sc_todos = len(juntando_dados[(juntando_dados.uf == 'SC') >= 1 ])


acidentes_mt_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'MT') >= 1 ])
acidentes_mt_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'MT') >= 1 ])
acidentes_mt_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'MT') >= 1 ])
acidentes_mt_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'MT') >= 1 ])
acidentes_mt_todos = len(juntando_dados[(juntando_dados.uf == 'MT') >= 1 ])


acidentes_sp_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'SP') >= 1 ])
acidentes_sp_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'SP') >= 1 ])
acidentes_sp_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'SP') >= 1 ])
acidentes_sp_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'SP') >= 1 ])
acidentes_sp_todos = len(juntando_dados[(juntando_dados.uf == 'SP') >= 1 ])


acidentes_df_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'DF') >= 1 ])
acidentes_df_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'DF') >= 1 ])
acidentes_df_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'DF') >= 1 ])
acidentes_df_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'DF') >= 1 ])
acidentes_df_todos = len(juntando_dados[(juntando_dados.uf == 'DF') >= 1 ])


acidentes_pr_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'PR') >= 1 ])
acidentes_pr_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'PR') >= 1 ])
acidentes_pr_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'PR') >= 1 ])
acidentes_pr_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'PR') >= 1 ])
acidentes_pr_todos = len(juntando_dados[(juntando_dados.uf == 'PR') >= 1 ])

acidentes_rn_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'RN') >= 1 ])
acidentes_rn_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'RN') >= 1 ])
acidentes_rn_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'RN') >= 1 ])
acidentes_rn_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'RN') >= 1 ])
acidentes_rn_todos = len(juntando_dados[(juntando_dados.uf == 'RN') >= 1 ])


acidentes_mg_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'MG') >= 1 ])
acidentes_mg_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'MG') >= 1 ])
acidentes_mg_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'MG') >= 1 ])
acidentes_mg_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'MG') >= 1 ])
acidentes_mg_todos = len(juntando_dados[(juntando_dados.uf == 'MG') >= 1 ])


acidentes_pe_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'PE') >= 1 ])
acidentes_pe_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'PE') >= 1 ])
acidentes_pe_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'PE') >= 1 ])
acidentes_pe_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'PE') >= 1 ])
acidentes_pe_todos = len(juntando_dados[(juntando_dados.uf == 'PE') >= 1 ])


acidentes_al_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'AL') >= 1 ])
acidentes_al_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'AL') >= 1 ])
acidentes_al_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'AL') >= 1 ])
acidentes_al_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'AL') >= 1 ])
acidentes_al_todos = len(juntando_dados[(juntando_dados.uf =='AL') >= 1 ])


acidentes_ms_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'MS') >= 1 ])
acidentes_ms_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'MS') >= 1 ])
acidentes_ms_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'MS') >= 1 ])
acidentes_ms_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'MS') >= 1 ])
acidentes_ms_todos = len(juntando_dados[(juntando_dados.uf =='MS') >= 1 ])


acidentes_go_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'GO') >= 1 ])
acidentes_go_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'GO') >= 1 ])
acidentes_go_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'GO') >= 1 ])
acidentes_go_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'GO') >= 1 ])
acidentes_go_todos = len(juntando_dados[(juntando_dados.uf =='GO') >= 1 ])


acidentes_ma_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'MA') >= 1 ])
acidentes_ma_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'MA') >= 1 ])
acidentes_ma_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'MA') >= 1 ])
acidentes_ma_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'MA') >= 1 ])
acidentes_ma_todos = len(juntando_dados[(juntando_dados.uf =='MA') >= 1 ])


acidentes_ba_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'BA') >= 1 ])
acidentes_ba_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'BA') >= 1 ])
acidentes_ba_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'BA') >= 1 ])
acidentes_ba_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'BA') >= 1 ])
acidentes_ba_todos = len(juntando_dados[(juntando_dados.uf =='BA') >= 1 ])


acidentes_to_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'TO') >= 1 ])
acidentes_to_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'TO') >= 1 ])
acidentes_to_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'TO') >= 1 ])
acidentes_to_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'TO') >= 1 ])
acidentes_to_todos = len(juntando_dados[(juntando_dados.uf =='TO') >= 1 ])


acidentes_pi_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'PI') >= 1 ])
acidentes_pi_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'PI') >= 1 ])
acidentes_pi_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'PI') >= 1 ])
acidentes_pi_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'PI') >= 1 ])
acidentes_pi_todos = len(juntando_dados[(juntando_dados.uf =='PI') >= 1 ])
                         

acidentes_ro_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'RO') >= 1 ])
acidentes_ro_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'RO') >= 1 ])
acidentes_ro_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'RO') >= 1 ])
acidentes_ro_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'RO') >= 1 ])
acidentes_ro_todos = len(juntando_dados[(juntando_dados.uf =='RO') >= 1 ])


acidentes_ce_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'CE') >= 1 ])
acidentes_ce_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'CE') >= 1 ])
acidentes_ce_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'CE') >= 1 ])
acidentes_ce_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'CE') >= 1 ])
acidentes_ce_todos = len(juntando_dados[(juntando_dados.uf =='CE') >= 1 ])


acidentes_se_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'SE') >= 1 ])
acidentes_se_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'SE') >= 1 ])
acidentes_se_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'SE') >= 1 ])
acidentes_se_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'SE') >= 1 ])
acidentes_se_todos = len(juntando_dados[(juntando_dados.uf =='SE') >= 1 ])


acidentes_pa_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'PA') >= 1 ])
acidentes_pa_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'PA') >= 1 ])
acidentes_pa_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'PA') >= 1 ])
acidentes_pa_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'PA') >= 1 ])
acidentes_pa_todos = len(juntando_dados[(juntando_dados.uf =='PA') >= 1 ])


acidentes_es_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'ES') >= 1 ])
acidentes_es_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'ES') >= 1 ])
acidentes_es_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'ES') >= 1 ])
acidentes_es_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'ES') >= 1 ])
acidentes_es_todos = len(juntando_dados[(juntando_dados.uf =='ES') >= 1 ])


acidentes_rr_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'RR') >= 1 ])
acidentes_rr_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'RR') >= 1 ])
acidentes_rr_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'RR') >= 1 ])
acidentes_rr_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'RR') >= 1 ])
acidentes_rr_todos = len(juntando_dados[(juntando_dados.uf =='RR') >= 1 ])

acidentes_am_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'AM') >= 1 ])
acidentes_am_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'AM') >= 1 ])
acidentes_am_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'AM') >= 1 ])
acidentes_am_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'AM') >= 1 ])
acidentes_am_todos = len(juntando_dados[(juntando_dados.uf =='AM') >= 1 ])


acidentes_ap_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'AP') >= 1 ])
acidentes_ap_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'AP') >= 1 ])
acidentes_ap_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'AP') >= 1 ])
acidentes_ap_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'AP') >= 1 ])
acidentes_ap_todos = len(juntando_dados[(juntando_dados.uf =='AP') >= 1 ])


acidentes_ac_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'AC') >= 1 ])
acidentes_ac_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'AC') >= 1 ])
acidentes_ac_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'AC') >= 1 ])
acidentes_ac_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'AC') >= 1 ])
acidentes_ac_todos = len(juntando_dados[(juntando_dados.uf =='AC') >= 1 ])

acidentes_rs_2018 = len(dados_prf_2018[(dados_prf_2018.uf == 'RS') >= 1 ])
acidentes_rs_2019 = len(dados_prf_2019[(dados_prf_2019.uf == 'RS') >= 1 ])
acidentes_rs_2020 = len(dados_prf_2020[(dados_prf_2020.uf == 'RS') >= 1 ])
acidentes_rs_2021 = len(dados_prf_2021[(dados_prf_2021.uf == 'RS') >= 1 ])
acidentes_rs_todos = len(juntando_dados[(juntando_dados.uf =='RS') >= 1 ])


In [8]:
estados = ['AC','AP','AM','RR','ES','PA',"SE",'CE','RO','PI','TO','BA','MA','GO','MS','AL','PE',
                    'MG','RN','PR','PB','MT','DF','SP','RS','SC','RJ']

In [9]:
ano_2018_por_estado = pd.DataFrame(pd.Series([acidentes_ac_2018,acidentes_ap_2018,acidentes_am_2018,
                                acidentes_rr_2018,acidentes_es_2018,acidentes_pa_2018,
                                acidentes_se_2018,acidentes_ce_2018,acidentes_ro_2018,
                                acidentes_pi_2018,acidentes_to_2018,acidentes_ba_2018,
                                acidentes_ma_2018,acidentes_go_2018,acidentes_ms_2018,
                                acidentes_al_2018,acidentes_pe_2018,acidentes_mg_2018,
                                acidentes_rn_2018,acidentes_pr_2018,acidentes_pb_2018,acidentes_mt_2018,
                                acidentes_df_2018,acidentes_sp_2018,acidentes_rs_2018,
                                acidentes_sc_2018,acidentes_rj_2018]))
ano_2018_por_estado.set_axis(["Ano_2018"], axis='columns', inplace=True)
ano_2018_por_estado['estados'] = estados
ano_2018_por_estado = ano_2018_por_estado.set_index('estados')

ano_2019_por_estado = pd.DataFrame(pd.Series([acidentes_ac_2019,acidentes_ap_2019,acidentes_am_2019,
                                acidentes_rr_2019,acidentes_es_2019,acidentes_pa_2019,
                                acidentes_se_2019,acidentes_ce_2019,acidentes_ro_2019,
                                acidentes_pi_2019,acidentes_to_2019,acidentes_ba_2019,
                                acidentes_ma_2019,acidentes_go_2019,acidentes_ms_2019,
                                acidentes_al_2019,acidentes_pe_2019,acidentes_mg_2019,
                                acidentes_rn_2019,acidentes_pr_2019,acidentes_pb_2019,acidentes_mt_2019,
                                acidentes_df_2019,acidentes_sp_2019,acidentes_rs_2019,
                                acidentes_sc_2019,acidentes_rj_2019]))
ano_2019_por_estado.set_axis(["Ano_2019"], axis='columns', inplace=True)
ano_2019_por_estado['estados'] = estados
ano_2019_por_estado = ano_2019_por_estado.set_index('estados')


ano_2020_por_estado = pd.DataFrame(pd.Series([acidentes_ac_2020,acidentes_ap_2020,acidentes_am_2020,
                                acidentes_rr_2020,acidentes_es_2020,acidentes_pa_2020,
                                acidentes_se_2020,acidentes_ce_2020,acidentes_ro_2020,
                                acidentes_pi_2020,acidentes_to_2020,acidentes_ba_2020,
                                acidentes_ma_2020,acidentes_go_2020,acidentes_ms_2020,
                                acidentes_al_2020,acidentes_pe_2020,acidentes_mg_2020,
                                acidentes_rn_2020,acidentes_pr_2020,acidentes_pb_2020,acidentes_mt_2020,
                                acidentes_df_2020,acidentes_sp_2020,acidentes_rs_2020,
                                acidentes_sc_2020,acidentes_rj_2020]))
ano_2020_por_estado.set_axis(["Ano_2020"], axis='columns', inplace=True)
ano_2020_por_estado['estados'] = estados
ano_2020_por_estado = ano_2020_por_estado.set_index('estados')


ano_2021_por_estado = pd.DataFrame(pd.Series([acidentes_ac_2021,acidentes_ap_2021,acidentes_am_2021,
                                acidentes_rr_2021,acidentes_es_2021,acidentes_pa_2021,
                                acidentes_se_2021,acidentes_ce_2021,acidentes_ro_2021,
                                acidentes_pi_2021,acidentes_to_2021,acidentes_ba_2021,
                                acidentes_ma_2021,acidentes_go_2021,acidentes_ms_2021,
                                acidentes_al_2021,acidentes_pe_2021,acidentes_mg_2021,
                                acidentes_rn_2021,acidentes_pr_2021,acidentes_pb_2021,acidentes_mt_2021,
                                acidentes_df_2021,acidentes_sp_2021,acidentes_rs_2021,
                                acidentes_sc_2021,acidentes_rj_2021]))
ano_2021_por_estado.set_axis(["Ano_2021"], axis='columns', inplace=True)
ano_2021_por_estado['estados'] = estados
ano_2021_por_estado = ano_2021_por_estado.set_index('estados')



In [10]:
ano_por_estado = pd.concat([ano_2018_por_estado, ano_2019_por_estado, ano_2020_por_estado,
                            ano_2021_por_estado], axis = 1)
ano_por_estado

,Ano_2018,Ano_2019,Ano_2020,Ano_2021
estados,,,,
AC,238,262,300,255
AP,173,128,155,170
AM,124,102,138,93
RR,236,237,250,268
ES,2642,2846,2534,2525
PA,1260,823,963,844
SE,559,612,575,511
CE,1697,1694,1572,1543
RO,1630,1536,1486,1419


In [11]:
todos_os_dados = juntando_dados.groupby(['municipio']).size().reset_index(name = 'Acidentes')
todos_os_dados = todos_os_dados.sort_values(by = 'Acidentes', ascending=False)
todos_os_dados.head(1000)

,municipio,Acidentes
253,BRASILIA,3979
551,CURITIBA,3966
1681,SAO JOSE,3012
743,GUARULHOS,2782
1246,PALHOCA,2496
...,...,...
763,IBEMA,47
1242,PADRE CARVALHO,47
144,AURORA DO PARA,47
958,JUPI,47


In [12]:
lista_ranking = []
for i in range(1, 100+1):
    lista_ranking.append(i)

In [13]:
x1 = ano_por_estado.index
x1

Index(['AC', 'AP', 'AM', 'RR', 'ES', 'PA', 'SE', 'CE', 'RO', 'PI', 'TO', 'BA',
       'MA', 'GO', 'MS', 'AL', 'PE', 'MG', 'RN', 'PR', 'PB', 'MT', 'DF', 'SP',
       'RS', 'SC', 'RJ'],
      dtype='object', name='estados')

In [14]:
dia = juntando_dados.groupby(['dia_semana']).size().reset_index(name ='por_dia')
semana_todos = dia.sort_values(by ='por_dia', ascending = False )

dia18 = dados_prf_2018.groupby(['dia_semana']).size().reset_index(name ='por_dia')
semana_18 = dia.sort_values(by ='por_dia', ascending = False )

dia19 = dados_prf_2019.groupby(['dia_semana']).size().reset_index(name ='por_dia')
semana_19 = dia.sort_values(by ='por_dia', ascending = False )

dia20 = dados_prf_2020.groupby(['dia_semana']).size().reset_index(name ='por_dia')
semana_20 = dia.sort_values(by ='por_dia', ascending = False )

dia21 = dados_prf_2021.groupby(['dia_semana']).size().reset_index(name ='por_dia')
semana_21 = dia.sort_values(by ='por_dia', ascending = False )
dia_semana = ["domingo", "quarta-feira" ,"quinta-feira" ,"segunda-feira" ,"sexta-feira", "sábado ,terça-feira"]


In [15]:
# manipulação de dados para analise acidente durantes e antes da pandemia e porcentagens
antes_da_pandemia = len(pd.concat([dados_prf_2018,dados_prf_2019]))
durante_pandemia = len(pd.concat([dados_prf_2020,dados_prf_2021]))
antes = antes_da_pandemia /2
durante = durante_pandemia /2
diferença = antes - durante

# porcentagem de aumento ou diminição de casos referente ao ano anterior
porcentagem18_19 = (((len(dados_prf_2018)-len(dados_prf_2019))/(len(dados_prf_2018)))*100)
porcentagem19_20 = (((len(dados_prf_2019)-len(dados_prf_2020))/(len(dados_prf_2019)))*100)
porcentagem20_21 = (((len(dados_prf_2020)-len(dados_prf_2021))/(len(dados_prf_2020)))*100)

porcentagem18_19_alg = round(porcentagem18_19,2)
porcentagem19_20_alg = round(porcentagem19_20,2)
porcentagem20_21_alg = round(porcentagem20_21,2)

porcentagem = pd.DataFrame([porcentagem18_19_alg,porcentagem19_20_alg ,porcentagem20_21_alg],
                           index=['Porcentagem de 2019 referente 2018','Porcentagem de 2020 referente 2019',
                                 'Porcentagem de 2021 referente 2020']).rename(columns={0:'Porcentagem %'})
porcentagem


,Porcentagem %
Porcentagem de 2019 referente 2018,2.56
Porcentagem de 2020 referente 2019,5.89
Porcentagem de 2021 referente 2020,-1.51


In [16]:
dia_anos = pd.concat([semana_todos,semana_18,semana_19,semana_20,semana_21],
                     keys = ['Todos','2018', '2019', '2020', '2021'],axis= 1 )
dia_anos

Todos                   2018                   2019          \
      dia_semana por_dia     dia_semana por_dia     dia_semana por_dia   
5         sábado   43907         sábado   43907         sábado   43907   
0        domingo   43843        domingo   43843        domingo   43843   
4    sexta-feira   40988    sexta-feira   40988    sexta-feira   40988   
3  segunda-feira   35737  segunda-feira   35737  segunda-feira   35737   
2   quinta-feira   34487   quinta-feira   34487   quinta-feira   34487   
1   quarta-feira   33371   quarta-feira   33371   quarta-feira   33371   
6    terça-feira   32670    terça-feira   32670    terça-feira   32670   

            2020                   2021          
      dia_semana por_dia     dia_semana por_dia  
5         sábado   43907         sábado   43907  
0        domingo   43843        domingo   43843  
4    sexta-feira   40988    sexta-feira   40988  
3  segunda-feira   35737  segunda-feira   35737  
2   quinta-feira   34487   quinta-feira   34487  
1   quarta-feira   33371   quarta-feira   33371  
6    terça-feira   32670    terça-feira   32670

In [17]:
#acidentes apenas na paraiba


dados_prf_2018
dados_prf_2019
dados_prf_2020
dados_prf_2021

paraiba_2018=dados_prf_2018[dados_prf_2018['uf']=='PB']
paraiba_2019=dados_prf_2019[dados_prf_2019['uf']=='PB']
paraiba_2020=dados_prf_2020[dados_prf_2020['uf']=='PB']
paraiba_2021=dados_prf_2021[dados_prf_2021['uf']=='PB']

aci18 = len(paraiba_2018.index)
aci19 = len(paraiba_2019.index)
aci20 = len(paraiba_2020.index)
aci21 = len(paraiba_2021.index)
data = {
    'Anos': ['2018', '2019', '2020', '2021'],
    'Acidentes': [aci18,aci19,aci20,aci21]}
acid = pd.DataFrame(data)
med_paraiba = acid['Acidentes'].mean()


#acidentes apenas no rio de janeiro


dados_prf_2018
dados_prf_2019
dados_prf_2020
dados_prf_2021

rio_2018=dados_prf_2018[dados_prf_2018['uf']=='RJ']
rio_2019=dados_prf_2019[dados_prf_2019['uf']=='RJ']
rio_2020=dados_prf_2020[dados_prf_2020['uf']=='RJ']
rio_2021=dados_prf_2021[dados_prf_2021['uf']=='RJ']

aci18 = len(rio_2018.index)
aci19 = len(rio_2019.index)
aci20 = len(rio_2020.index)
aci21 = len(rio_2021.index)
data = {
    'Anos': ['2018', '2019', '2020', '2021'],
    'Acidentes': [aci18,aci19,aci20,aci21]}
acid = pd.DataFrame(data)
med_rj = acid['Acidentes'].mean()


#acidentes apenas em sp


dados_prf_2018
dados_prf_2019
dados_prf_2020
dados_prf_2021

SP_2018=dados_prf_2018[dados_prf_2018['uf']=='SP']
SP_2019=dados_prf_2019[dados_prf_2019['uf']=='SP']
SP_2020=dados_prf_2020[dados_prf_2020['uf']=='SP']
SP_2021=dados_prf_2021[dados_prf_2021['uf']=='SP']

aci18 = len(SP_2018.index)
aci19 = len(SP_2019.index)
aci20 = len(SP_2020.index)
aci21 = len(SP_2021.index)
data = {
    'Anos': ['2018', '2019', '2020', '2021'],
    'Acidentes': [aci18,aci19,aci20,aci21]}
acid = pd.DataFrame(data)
med_sp = acid['Acidentes'].mean()





In [21]:
app = Dash(__name__)

colors = {
    'background': '#111111',
    'text': '#7FDBFF'
}


fig = px.bar(acidentes_ano, x="Anos", y="Acidentes",color = "Anos", barmode="group")
fig2 = px.line(ano_por_estado, x=x1, y="Ano_2018", text="Ano_2018")
fig3 = px.bar(todos_os_dados.head(10), x="municipio", y="Acidentes",color = "Acidentes", barmode="group")
fig4 = px.pie(acidentes_ano, values='Acidentes', names='Anos')
fig4.update_layout(title_text="Porcentagem de aumento ou diminuição em relação aos outros anos",
    title_font_size=20)
fig4.update_layout(template="plotly_dark")
labels = ['SP','RJ','PB']
values = [med_sp, med_rj, med_paraiba]
fig5 = go.Figure(data=[go.Pie(labels=labels, values=values, pull=[0, 0, 0.2])])
fig5.update_layout(title_text="Comparando as três cidades e vendo qual é que tem mais acidentes comparada as outras",
              title_font_size=20)
fig5.update_layout(template="plotly_dark")
fig6 = px.pie(semana_todos, values='por_dia', names="dia_semana")
fig6.update_layout(title_text="Comparando as três cidades e vendo qual é que tem mais acidentes comparada as outras",
              title_font_size=20)
fig6.update_layout(template="plotly_dark")

fig.update_layout(
    plot_bgcolor=colors['background'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']


)

app.layout = html.Div(style={'backgroundColor': colors['background']}, children=[
    html.H1(
        children='Dados da Polícia Rodoviária Federal',
        style={
            'textAlign': 'center',
            'color': colors['text']
        }
    ),

    html.Div(children='Quantos acidentes ocorrem nos anos de 2018 a 2021.', style={
        'textAlign': 'center',
        'color': colors['text']
    
    }),
    
    
    dcc.Dropdown(['2018', '2019', '2020', '2021', 'Todos'],
                  value='Todos',
                  id ="lista de anos"),
    dcc.Graph(
        id='ACIDENTE ANOS',
        figure=fig
    ),
    html.Div(children='Quantos acidentes ocorrem em cada estado nos anos de 2018 a 2021.', style={
        'textAlign': 'center',
        'color': colors['text']
    
    }),
    
    dcc.Dropdown(['Ano_2018', 'Ano_2019', 'Ano_2020', 'Ano_2021',],
                  value='Ano_2018',
                  id ="anos"),
    
    dcc.Graph(
        id='ACIDENTE ANOS2',
        figure=fig2
    
    ),
    html.Div(children='Ranking de municipios por acidente de 1 a 100.', style={
        'textAlign': 'center',
        'color': colors['text']
    
    }),
    
    dcc.Dropdown(lista_ranking,
                  value=10,
                  id ="MUNICIPIOS"),
    dcc.Graph(
        id='RANKING',
        figure=fig3
    ),
    html.Div(children='Porcentagem de aumento ou diminuição de acidentes', style={
        'textAlign': 'center',
        'color': colors['text']
    
    }),
    
    dcc.Graph(
        id='Porcentagem de ano',
        figure=fig4
    ),
    html.Div(children='Comparação de Três cidades', style={
        'textAlign': 'center',
        'color': colors['text']
    
    }),
    
    dcc.Graph(
        id='comparação',
        figure=fig5
    ),
    html.Div(children='Acidentes no dia da semana de todos os anos e de 2018 a 2021', style={
            'textAlign': 'center',
            'color': colors['text']
    
    }),
    
    dcc.Dropdown(['2018', '2019', '2020', '2021', 'Todos'],
                  value="Todos",
                  id ="qual dia ano"),
    dcc.Graph(
        id='dia semana',
        figure=fig6
    )
])

@app.callback(
    Output('ACIDENTE ANOS', 'figure'),
    Input('lista de anos', 'value')
)
def update_output(value):
    if value == "Todos":    
        fig = px.bar(acidentes_ano, x="Anos", y="Acidentes",color = "Anos", text="Acidentes",barmode="group")
        fig.update_layout(template="plotly_dark")
    else:
        filtrada_aci_anos = acidentes_ano.loc[acidentes_ano["Anos"]==value, :]
        fig = px.bar(filtrada_aci_anos, x="Anos", y="Acidentes",color = "Anos", text="Acidentes",barmode="group")
        fig.update_layout(template="plotly_dark")
    return fig


@app.callback(
    Output('ACIDENTE ANOS2', 'figure'),
    Input('anos', 'value')
)

def update_output(value):
    fig2 = px.line(ano_por_estado, x=x1, y= value, text=value)
    fig2.update_layout(title_text="Quantidade de acidentes por Estado e Ano",
              title_font_size=20)
    fig2.update_traces(textposition="bottom right")
    fig2.update_layout(template="plotly_dark")
    
    return fig2


@app.callback(
    Output('RANKING', 'figure'),
    Input('MUNICIPIOS', 'value')
)
def update_output(value):
    fig3 = px.bar(todos_os_dados.head(value), x="municipio", y="Acidentes",text="Acidentes",color = "Acidentes", barmode="group")
    fig3.update_layout(title_text="Quantidade de acidentes por Estado e Ano",
              title_font_size=20)
    fig3.update_layout(template="plotly_dark")
    return fig3

@app.callback(
    Output('dia semana', 'figure'),
    Input('qual dia ano', 'value')
)
def update_output(value):
    if value == "Todos":
        fig6 = px.pie(semana_todos, values='por_dia', names="dia_semana")
        fig6.update_layout(title_text="dias da semana que teve mais acidentes",
              title_font_size=20)
        fig6.update_layout(template="plotly_dark")
    elif value == "2018":
        fig6 = px.pie(dia18, values='por_dia', names="dia_semana")
        fig6.update_layout(title_text="dias da semana que teve mais acidentes",
              title_font_size=20)
        fig6.update_layout(template="plotly_dark")
    elif value == "2019":
        fig6 = px.pie(dia19, values='por_dia', names="dia_semana")
        fig6.update_layout(title_text="dias da semana que teve mais acidentes",
              title_font_size=20)
        fig6.update_layout(template="plotly_dark")
    elif value == "2020":
        fig6 = px.pie(dia20, values='por_dia', names="dia_semana")
        fig6.update_layout(title_text="dias da semana que teve mais acidentes",
              title_font_size=20)
        fig6.update_layout(template="plotly_dark")
    elif value == "2021":
        fig6 = px.pie(dia21, values='por_dia', names="dia_semana")
        fig6.update_layout(title_text="dias da semana que teve mais acidentes",
              title_font_size=20)
        fig6.update_layout(template="plotly_dark")
    return fig6



if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
